<a href="https://colab.research.google.com/github/TRxcha/TELECOMX/blob/main/TelecomX_BR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracão

# Desafio de Análise de Evasão de Clientes (Churn)

## 🎯 Introdução
O objetivo deste desafio foi analisar dados de clientes de uma empresa de telecomunicações a fim de identificar padrões relacionados à **evasão de clientes (Churn)**.  
A evasão é um dos principais problemas do setor de telecom, já que conquistar novos clientes é mais caro do que reter os atuais.

In [99]:
import pandas as pd
import numpy as np
import plotly.express as px

In [100]:
url = 'https://raw.githubusercontent.com/alura-cursos/challenge2-data-science/refs/heads/main/TelecomX_Data.json'
df_bruto = pd.read_json(url)

df_bruto.info()
df_bruto.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [101]:
# Normalizando o DataFrame
df = pd.concat([
    df_bruto[['customerID', 'Churn']],
    pd.json_normalize(df_bruto['customer'], sep='_'),
    pd.json_normalize(df_bruto['phone'], sep='_'),
    pd.json_normalize(df_bruto ['internet'], sep='_'),
    pd.json_normalize(df_bruto['account'], sep='_')
], axis=1)

df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


#🔧 Transformação

In [102]:
# Verificando se há valores nulos e eliminando duplicatas
df.isnull().sum()
df.duplicated(subset=['customerID'])
df = df.drop_duplicates(subset=['customerID'])
len(df)

df["Churn"] = df["Churn"].astype(str).str.strip().str.title()
df["Churn"] = df["Churn"].replace({"Nan": None, "": None})

In [103]:
# Convertendo colunas para o tipo de dado adequado e eliminando a custumerID
col_num = ['SeniorCitizen', 'tenure']

df[col_num] = df[col_num].astype(np.int64)
df['Charges_Total'] = (df['Charges_Total'].str.replace(',', '').str.strip().replace('', '0').astype(np.float64))
df.drop('customerID', axis=1, inplace=True)

df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Churn             7043 non-null   object 
 1   gender            7267 non-null   object 
 2   SeniorCitizen     7267 non-null   int64  
 3   Partner           7267 non-null   object 
 4   Dependents        7267 non-null   object 
 5   tenure            7267 non-null   int64  
 6   PhoneService      7267 non-null   object 
 7   MultipleLines     7267 non-null   object 
 8   InternetService   7267 non-null   object 
 9   OnlineSecurity    7267 non-null   object 
 10  OnlineBackup      7267 non-null   object 
 11  DeviceProtection  7267 non-null   object 
 12  TechSupport       7267 non-null   object 
 13  StreamingTV       7267 non-null   object 
 14  StreamingMovies   7267 non-null   object 
 15  Contract          7267 non-null   object 
 16  PaperlessBilling  7267 non-null   object 


,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
0,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30
1,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40
2,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40


In [104]:
# Criando a coluna de contas diárias
df['Contas_Diarias'] = df['Charges_Monthly'] / 30
df

#📊 Carga e análise

In [105]:
df_clean = df.dropna(subset=["Churn"])
fig1 = px.histogram(df_clean, x="Churn", title="Distribuição do Churn", color="Churn")
fig1.show()

In [107]:
fig2 = px.histogram(df_clean, x="gender", color="Churn", barmode="group",
                    title="Churn por Gênero")
fig2.show()

In [108]:
fig3 = px.histogram(df_clean, x="Contract", color="Churn", barmode="group",
                    title="Churn por Tipo de Contrato")
fig3.show()

In [109]:
fig1 = px.box(df_clean, x="Churn", y="tenure", color="Churn",
              title="Distribuição do Tempo de Contrato por Churn")
fig1.show()

In [110]:
fig4 = px.box(df_clean, x="Churn", y="Contas_Diarias", color="Churn",
              title="Distribuição de Contas Diárias por Churn")
fig4.show()

#📄Relatorio Final

## 🔍 Conclusões e Insights
- Clientes com **baixo tempo de contrato (tenure)** têm maior probabilidade de evasão.  
- Em geral, clientes que **pagam mais por mês (Charges_Monthly)** estão mais propensos ao churn, especialmente se não têm contratos de longo prazo.  
- O **total gasto (Charges_Total)** é menor entre churners, já que eles permanecem menos tempo.  
- A análise mostra perfis distintos de clientes que podem embasar estratégias de retenção.  

## 💡 Recomendações
1. **Onboarding reforçado para novos clientes** → suporte proativo nos primeiros meses.  
2. **Estratégias de precificação** → revisar cobranças mensais elevadas, oferecendo pacotes de maior valor percebido.  
3. **Alertas de risco** → monitorar clientes com baixa fidelização (tenure baixo + cobrança alta).  
4. **Ofertas personalizadas** → segmentar benefícios conforme perfil de consumo (`Charges_Total`).  
5. **Testes A/B de retenção** → validar estratégias como descontos progressivos e bônus por fidelidade.  